# Tagging and Extraction Using OpenAI functions

In [1]:
from typing import List
from pydantic import BaseModel, Field

class Tagging(BaseModel):
    """Tag the piece of text with particular info."""

    sentiment: str = Field(
        description="sentiment of text, should be `pos`, `neg`, or `neutral`"
    )
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [4]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

convert_pydantic_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language'],
  'type': 'object'}}

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI(temperature=0)

In [7]:
tagging_functions = [convert_pydantic_to_openai_function(Tagging)]

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Think carefully, and then tag the text as instructed"),
        ("user", "{input}"),
    ]
)

In [9]:
model_with_functions = model.bind(
    functions=tagging_functions, function_call={"name": "Tagging"}
)

In [10]:
tagging_chain = prompt | model_with_functions

In [12]:
from helper import show_response

show_response(tagging_chain.invoke({"input": "I love langchain"}))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"sentiment\":\"pos\",\"language\":\"en\"}",
      "name": "Tagging"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 10,
      "prompt_tokens": 108,
      "total_tokens": 118,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxILJPfNff5XPobVyED7J8AK8GNyk",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--2211d44e-eaac-41f9-bec0-b6962aed466f-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "

In [13]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentiment":"neg","language":"it"}', 'name': 'Tagging'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxILLEpRw43Z19pqhLbv2i8TWtqU3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--cd24f619-00b5-4d0c-b8be-ac3efea03251-0', usage_metadata={'input_tokens': 111, 'output_tokens': 10, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [15]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [16]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [17]:
from typing import Optional


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [18]:
class Information(BaseModel):
    """Information to extract."""

    people: List[Person] = Field(description="List of info about people")

In [19]:
convert_pydantic_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [20]:
extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_model = model.bind(
    functions=extraction_functions, function_call={"name": "Information"}
)

In [21]:
show_response(extraction_model.invoke("Joe is 30, his mom is Martha"))

{
  "content": "",
  "additional_kwargs": {
    "function_call": {
      "arguments": "{\"people\":[{\"name\":\"Joe\",\"age\":30},{\"name\":\"Martha\",\"age\":null}]}",
      "name": "Information"
    },
    "refusal": null
  },
  "response_metadata": {
    "token_usage": {
      "completion_tokens": 21,
      "prompt_tokens": 95,
      "total_tokens": 116,
      "completion_tokens_details": {
        "accepted_prediction_tokens": 0,
        "audio_tokens": 0,
        "reasoning_tokens": 0,
        "rejected_prediction_tokens": 0
      },
      "prompt_tokens_details": {
        "audio_tokens": 0,
        "cached_tokens": 0
      }
    },
    "model_name": "gpt-3.5-turbo-0125",
    "system_fingerprint": null,
    "id": "chatcmpl-BxILhbeNyrjbEjIu8GbFss19REvDr",
    "service_tier": "default",
    "finish_reason": "stop",
    "logprobs": null
  },
  "type": "ai",
  "name": null,
  "id": "run--1d53bb43-0088-4ea5-a6b7-c38276552261-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Extract the relevant information, if not explicitly provided do not guess. Extract partial info. Do not add None values."  # noqa: E501,
        ),
        ("human", "{input}"),
    ]
)

In [33]:
extraction_chain = prompt | extraction_model

In [34]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha"}]}', 'name': 'Information'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 118, 'total_tokens': 135, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BxIMsYwrny6zAx50zaotJEc24zoUp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--861afcd6-2c21-49d6-bccb-a770b53c21d6-0', usage_metadata={'input_tokens': 118, 'output_tokens': 17, 'total_tokens': 135, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [36]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

In [37]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")
)

In [38]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [45]:
from langchain.document_loaders import WebBaseLoader
import os

# Set the USER_AGENT environment variable to avoid warnings
os.environ["USER_AGENT"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

# Set up the web loader with a proper user agent to avoid warnings
loader = WebBaseLoader(
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    header_template={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
)

In [46]:
# Load the documents from the web page
documents = loader.load()

doc = documents[0]

In [47]:
page_content = doc.page_content[:10000]

In [48]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.


In [49]:
class Overview(BaseModel):
    """Overview of a section of text."""

    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(
        description="Provide the language that the content is written in."
    )
    keywords: str = Field(description="Provide keywords related to the content.")

In [50]:
overview_tagging_function = [convert_pydantic_to_openai_function(Overview)]
tagging_model = model.bind(
    functions=overview_tagging_function, function_call={"name": "Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [51]:
tagging_chain.invoke({"input": page_content})

{'summary': 'Overview of LLM-powered autonomous agents system with components like Planning, Memory, and Tool Use. Discusses task decomposition, self-reflection, and challenges in building autonomous agents with LLM.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, challenges'}

In [52]:
class Paper(BaseModel):
    """Information about papers mentioned."""

    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""

    papers: List[Paper]

In [53]:
paper_extraction_function = [convert_pydantic_to_openai_function(Info)]
extraction_model = model.bind(
    functions=paper_extraction_function, function_call={"name": "Info"}
)
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [54]:
extraction_chain.invoke({"input": page_content})

[{'title': 'LLM Powered Autonomous Agents', 'author': 'Lilian Weng'}]

In [55]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article follow by its author. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

In [56]:
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [57]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)',
  'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al. 2023'}]

In [58]:
extraction_chain.invoke({"input": "hi"})

[{'title': 'Paper A', 'author': 'Author A'},
 {'title': 'Paper B', 'author': 'Author B'}]

In [59]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [60]:
splits = text_splitter.split_text(doc.page_content)

In [61]:
len(splits)

15

In [62]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [63]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [64]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log

















|






Posts




Archive




Search




Tags




FAQ









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent 

In [65]:
from langchain.schema.runnable import RunnableLambda

In [66]:
prep = RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])

In [67]:
prep.invoke("hi")

[{'input': 'hi'}]

In [68]:
chain = prep | extraction_chain.map() | flatten

In [69]:
chain.invoke(doc.page_content)

[{'title': 'AutoGPT', 'author': None},
 {'title': 'GPT-Engineer', 'author': None},
 {'title': 'BabyAGI', 'author': None},
 {'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Chain of Hindsight (CoH)', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation (AD)', 'author': 'Laskin et al. 2023'},
 {'title': 'ED', 'author': None},
 {'title': 'RL^2', 'author': 'Duan et al. 2017'},
 {'title': 'Laskin et al. 2023', 'author': None},
 {'title': 'Miller 1956', 'author': None},
 {'title': 'Google Blog', 'author': None},
 {'title': 'MRKL', 'author': 'Karpas et al. 2022'},
 {'title': 'TALM', 'author': 'Parisi et al. 2022'},
 {'title': 'Toolformer', 'author': 'Schick et al. 2023'},
 {'title': 'HuggingGPT', 'author': 'Shen et al. 2023'},
 {'title': 'API-